# Filter Gene Lists
**Isobel Taylor-Hearn — Jan 2026**

**Purpose:**  
Filter differential expression Excel files to retain genes of interest, while flagging duplicated genes and explicitly recording genes absent from each sample.

**Inputs:**  
- Excel file of genes of interest (gene symbols in first column)  
- Folder of Excel files to be processed

**Outputs:**  
- Filtered CSV files with presence and duplication flags  
- CSV files listing duplicated genes per sample

In [5]:
import pandas as pd
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog

In [6]:
def filter_gene_list(excel_path, column_name = "HGNC.symbol"):
    """
    Load a gene list from an Excel file, report duplicates, and return
    a de-duplicated list of genes of interest.

    Parameters
    ----------
    excel_path : str
        Path to the Excel file containing the gene list. The first column
        is assumed to contain gene symbols.
    column_name : str, optional
        Name to assign to the gene symbol column in the output dataframe.
        Default is "HGNC.symbol".

    Returns
    -------
    unique_genes_of_interest : pandas.DataFrame
        DataFrame with a single column (`column_name`) containing unique
        gene symbols only.

    Side Effects
    ------------
    - Prints the total number of genes listed and how many are unique.
    - Prints gene symbols that appear more than once, along with their counts.

    Notes
    -----
    - Duplicate detection is performed on the full gene list before
      de-duplication.
    - Only the first column of the Excel file is used.
    """
    genes_of_interest = pd.read_excel(excel_path)
    genes_of_interest = genes_of_interest[genes_of_interest.columns[0]].to_frame(name=column_name)
    unique_genes_of_interest = genes_of_interest.drop_duplicates(subset=[column_name])
    print("You have listed {} genes of interest, of which {} are unique.".format(len(genes_of_interest), genes_of_interest[column_name].nunique()))
    dup_gene_names = (genes_of_interest[column_name].value_counts().loc[lambda s: s > 1])
    print(dup_gene_names)
    return unique_genes_of_interest

def process_folder_of_exels(folder_of_excels, unique_genes_of_interest, column_name = "HGNC.symbol"):
    """
    Process a folder of differential expression Excel files, keeping only
    rows matching a list of genes of interest, flagging repeated genes, and
    explicitly recording missing genes.

    Parameters
    ----------
    folder_of_excels : str
        Path to a directory containing Excel files to process.
    unique_genes_of_interest : pandas.DataFrame
        DataFrame containing a single column (`column_name`) of unique
        gene symbols of interest.
    column_name : str, optional
        Name of the gene symbol column to use in all dataframes.
        Default is "HGNC.symbol".

    Outputs
    -------
    For each Excel file in `folder_of_excels`:
    - A CSV listing all duplicated rows in the sample
      (`duplicates_in_<filename>.csv`)
    - A CSV containing:
        * all rows whose gene is in `unique_genes_of_interest`
        * a flag indicating whether the gene is repeated in the sample
        * explicit rows for genes of interest that are not present
      (`filtered_<filename>.csv`)

    Side Effects
    ------------
    - Prints summary statistics for each file:
        * total rows and number of unique genes
        * number of duplicated genes
        * number of genes of interest missing from the sample

    Notes
    -----
    - Duplicate genes in the sample are not collapsed; all rows are preserved.
    - Missing genes are represented as rows with `gene_present = False`
      and NaN values for all other columns.
    """
    excel_files = [f for f in os.listdir(folder_of_excels) if f.endswith('.xlsx')]
    for excel_file in excel_files:
        print(f"Processing file: {excel_file}")
            
        df = pd.read_excel(os.path.join(folder_of_excels, excel_file))
        print(df.shape)
        print("You have {} rows in the sample, of which {} HGNC symbols are unique.".format(len(df), df[column_name].nunique()))    
        dupes = (df[df[column_name].duplicated(keep=False)].sort_values(column_name))
        dupes.to_csv(os.path.join(folder_of_excels, f"duplicates_in_{excel_file.replace('.xlsx', '.csv')}"), index=False)
        
        filtered = df[ df[column_name].isin(unique_genes_of_interest[column_name])].copy()
        filtered["is_repeated_gene"] = (filtered[column_name].duplicated(keep=False))
        missing_genes = unique_genes_of_interest.loc[~unique_genes_of_interest[column_name].isin(df[column_name]), column_name]
        print("Of your genes of interest, {} are not present in the sample.".format(missing_genes.shape[0]))
        missing_df = pd.DataFrame({column_name: missing_genes,"is_repeated_gene": False,"gene_present": False})
        filtered["gene_present"] = True
        final_df = pd.concat([filtered, missing_df], ignore_index=True, sort=False)
        final_df.to_csv(os.path.join(folder_of_excels, f"filtered_{excel_file.replace('.xlsx', '.csv')}"), index=False)
        print("\n")


In [ ]:
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)
root.update() 

folder_of_excels = filedialog.askdirectory(parent=root, title="Select folder containing DE Excel files")
excel_path = filedialog.askopenfilename(parent=root, title="Select Gene List Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
root.destroy()

print("Folder:", folder_of_excels)
print("Gene list:", excel_path)

unique_genes_of_interest = filter_gene_list(excel_path)
process_folder_of_exels(folder_of_excels, unique_genes_of_interest)

Folder: Z:/Marina/Limfoma project/Bulk mRNA seq/DE excels
Gene list: Z:/Marina/Limfoma project/Bulk mRNA seq/Gene list.xlsx
You have listed 65 genes of interest, of which 59 are unique.
HGNC.symbol
ACKR1    2
CD34     2
CA4      2
GJA5     2
PLVAP    2
RGCC     2
Name: count, dtype: int64
Processing file: DE_HUVECLF_vs_HUVECTF_all.xlsx
(22087, 8)
You have 22087 rows in the sample, of which 18818 HGNC symbols are unique.
Of your genes of interest, 4 are not present in the sample.


Processing file: DE_HUVECLF_vs_tonsil_all.xlsx
(22087, 8)
You have 22087 rows in the sample, of which 18818 HGNC symbols are unique.
Of your genes of interest, 4 are not present in the sample.


Processing file: DE_HUVECTF_vs_tonsil_all.xlsx
(22087, 8)
You have 22087 rows in the sample, of which 18818 HGNC symbols are unique.
Of your genes of interest, 4 are not present in the sample.


